In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import sys
sys.path.insert(0, '/home/am2770/Github/cancer-low-data')
sys.path.insert(0, '/home/am2770/Github/cancer-low-data/src')
sys.path.insert(0, '/home/am2770/Github/cancer-low-data/notebooks')

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import wandb
import datetime
from utils import *
from _utils import filter_df_on_conditions, make_subplots
import seaborn as sns
from functools import partial

In [ ]:
datetime.datetime.now(datetime.timezone.utc).isoformat()

In [ ]:
api = wandb.Api(timeout=60)

In [ ]:
#### Get runs by tag
runs = api.runs('andreimargeloiu/low-data', {
	"tags": '8cb3b770',
	'State': 'finished'
})

#### Get one run
run = api.run('andreimargeloiu/low-data/rwzd9fud')

#### Get runs by date
runs = api.runs('andreimargeloiu/low-data', {
	"$and": [{
		'created_at': {
			"$lt": '2022-03-09T10',
			"$gt": '2022-03-07T17'
		},
		'config.embedding_type': 'nmf',
	}]
	'$or': [
		{'tags': '!!!!'},
		{'tags': '!!!!'},
		{'tags': '!!!!'},
		{
			"tags": '8e85015a',  # nmf size 50 
			'config.gamma': {'$eq': 0}
		}
	}
	],
	'State': 'finished',
})
print(len(runs))
runs = filter_runs(runs, return_dataframe=False)
print(f"Found {len(runs)} runs")
runs_df = transforms_runs_into_dataframe(runs)
print(runs_df.shape)

In [ ]:
### Optional commands
# add_tag_and_update(runs, TAG)

# - run.created_at
# - run.config.keys()
# - run.summary._json_dict['bestmodel_train/balanced_accuracy']

In [ ]:
runs_df.groupby(['dataset', 'embedding_type']).size()

In [ ]:
### Plot
pd.options.display.float_format = '{:,.2f}'.format

grouped = runs_df.groupby(['dataset', 'embedding_type'])
g1 = grouped.agg({'bestmodel_train/balanced_accuracy': ['mean', 'std']})*100
g2 = grouped.agg({'bestmodel_test/balanced_accuracy': ['mean', 'std']})*100
res = pd.concat([g1, g2], axis=1).round(decimals=2)
res

In [ ]:
# file_name = 'test'
# res.to_csv(f'/home/am2770/Github/cancer-low-data/compute_results/csv/{file_name}.csv')
# res.to_excel(f'/home/am2770/Github/cancer-low-data/compute_results/csv/{file_name}.xlsx')